In [9]:
import eventlet
import pandas as pd
from multiprocessing.dummy import Pool as ThreadPool
import requests
import lxml
from lxml import etree
from geopy.geocoders import Nominatim
import geopy
import pgeocode
import wikipediaapi


In [10]:
df = pd.read_csv('Database/Russian_cities.csv', dtype=str)
del df['Unnamed: 0']
df = df.loc[df['City'] != 'Щелкино']
df = df.reset_index(drop=True)

In [11]:
df

,Letter,City,Population
0,а,Астрахань,529793
1,а,Архангельск,346979
2,а,Ангарск,224630
3,а,Армавир,188960
4,а,Абакан,186797
...,...,...,...
1099,я,Яранск,15475
1100,я,Ясногорск,15410
1101,я,Ясный,15249
1102,я,Яхрома,14056


In [12]:
a = []
b = []
c = []
for i in df['City']:
    a.append(f"https://www.google.com/search?q={i.replace(' ', '+')}")
    b.append(f"https://yandex.ru/search/?text={i.replace(' ', '+')}")
    c.append(f"https://ru.wikipedia.org/wiki/{i.replace(' ', '_')}")

a = pd.Series(a, name='GoogleUrl')
b = pd.Series(b, name='YandexUrl')
df = pd.concat([df, a], axis=1)
df = pd.concat([df, b], axis=1)
df

,Letter,City,Population,GoogleUrl,YandexUrl
0,а,Астрахань,529793,https://www.google.com/search?q=Астрахань,https://yandex.ru/search/?text=Астрахань
1,а,Архангельск,346979,https://www.google.com/search?q=Архангельск,https://yandex.ru/search/?text=Архангельск
2,а,Ангарск,224630,https://www.google.com/search?q=Ангарск,https://yandex.ru/search/?text=Ангарск
3,а,Армавир,188960,https://www.google.com/search?q=Армавир,https://yandex.ru/search/?text=Армавир
4,а,Абакан,186797,https://www.google.com/search?q=Абакан,https://yandex.ru/search/?text=Абакан
...,...,...,...,...,...
1099,я,Яранск,15475,https://www.google.com/search?q=Яранск,https://yandex.ru/search/?text=Яранск
1100,я,Ясногорск,15410,https://www.google.com/search?q=Ясногорск,https://yandex.ru/search/?text=Ясногорск
1101,я,Ясный,15249,https://www.google.com/search?q=Ясный,https://yandex.ru/search/?text=Ясный
1102,я,Яхрома,14056,https://www.google.com/search?q=Яхрома,https://yandex.ru/search/?text=Яхрома


In [13]:
def checkurl(url):
    tf = True
    s = requests.get(url).text
    page = lxml.etree.fromstring(s, parser=lxml.etree.HTMLParser())
    for i in page.xpath('//*[@id="noarticletext"]/p'):
        s = ''.join(i.itertext())
        if 'нет статьи' in s:
            print(url + '\n', end='')
            tf = False
    if tf:
        return url
# pool = ThreadPool(20)
# results = list(pool.map(checkurl, c))
results = c
# results = list(map(checkurl, c))

# results

In [14]:
c = pd.Series(results, name='WikiUrl')
df = pd.concat([df, c], axis=1)
df
# //*[@class="infobox"]/tr[3]/td/span/span/a.href()
# //*[@id="mw-content-text"]/div[1]/table[1]/tr[3]/td/span/span/a.href()
# //*[@class="infobox-image"]/span/a
# //*[@class="wikidata-claim"]/span/a

,Letter,City,Population,GoogleUrl,YandexUrl,WikiUrl
0,а,Астрахань,529793,https://www.google.com/search?q=Астрахань,https://yandex.ru/search/?text=Астрахань,https://ru.wikipedia.org/wiki/Астрахань
1,а,Архангельск,346979,https://www.google.com/search?q=Архангельск,https://yandex.ru/search/?text=Архангельск,https://ru.wikipedia.org/wiki/Архангельск
2,а,Ангарск,224630,https://www.google.com/search?q=Ангарск,https://yandex.ru/search/?text=Ангарск,https://ru.wikipedia.org/wiki/Ангарск
3,а,Армавир,188960,https://www.google.com/search?q=Армавир,https://yandex.ru/search/?text=Армавир,https://ru.wikipedia.org/wiki/Армавир
4,а,Абакан,186797,https://www.google.com/search?q=Абакан,https://yandex.ru/search/?text=Абакан,https://ru.wikipedia.org/wiki/Абакан
...,...,...,...,...,...,...
1099,я,Яранск,15475,https://www.google.com/search?q=Яранск,https://yandex.ru/search/?text=Яранск,https://ru.wikipedia.org/wiki/Яранск
1100,я,Ясногорск,15410,https://www.google.com/search?q=Ясногорск,https://yandex.ru/search/?text=Ясногорск,https://ru.wikipedia.org/wiki/Ясногорск
1101,я,Ясный,15249,https://www.google.com/search?q=Ясный,https://yandex.ru/search/?text=Ясный,https://ru.wikipedia.org/wiki/Ясный
1102,я,Яхрома,14056,https://www.google.com/search?q=Яхрома,https://yandex.ru/search/?text=Яхрома,https://ru.wikipedia.org/wiki/Яхрома


In [15]:
df.loc[df['City'] == 'Алексеевка', 'WikiUrl'] = 'https://ru.wikipedia.org/wiki/Алексеевка_(город,_Белгородская_область)'
df.loc[df['City'] == 'Алексеевка', 'GoogleUrl'] = 'https://www.google.com/search?q=Город+Алексеевка+в+Белгородской+области'
df.loc[df['City'] == 'Алексеевка', 'YandexUrl'] = 'https://yandex.ru/search/?text=Город+Алексеевка+в+Белгородской+области'

df.loc[df['City'] == 'Буй', 'WikiUrl'] = 'https://ru.wikipedia.org/wiki/Буй_(город)'
df.loc[df['City'] == 'Буй', 'GoogleUrl'] = 'https://www.google.com/search?q=Буй+город'
df.loc[df['City'] == 'Буй', 'YandexUrl'] = 'https://yandex.ru/search/?text=Буй+город'

df.loc[df['City'] == 'Калач', 'WikiUrl'] = 'https://ru.wikipedia.org/wiki/Калач_(город)'
df.loc[df['City'] == 'Калач', 'GoogleUrl'] = 'https://www.google.com/search?q=Калач+город'
df.loc[df['City'] == 'Калач', 'YandexUrl'] = 'https://yandex.ru/search/?text=Калач+город'

df.loc[df['City'] == 'Донецк', 'WikiUrl'] = 'https://ru.wikipedia.org/wiki/Донецк_(Россия)'
df.loc[df['City'] == 'Донецк', 'GoogleUrl'] = 'https://www.google.com/search?q=Донецк+Россия'
df.loc[df['City'] == 'Донецк', 'YandexUrl'] = 'https://yandex.ru/search/?text=Донецк+Россия'

df.loc[df['City'] == 'Александровск', 'WikiUrl'] = 'https://ru.wikipedia.org/wiki/Александровск_(Пермский_край)'
df.loc[df['City'] == 'Александровск', 'GoogleUrl'] = 'https://www.google.com/search?q=Александровск+Пермский+край'
df.loc[df['City'] == 'Александровск', 'YandexUrl'] = 'https://yandex.ru/search/?text=Александровск+Пермский+край'

In [16]:
def checkurl(url):
    tf = True
    s = requests.get(url).text
    page = lxml.etree.fromstring(s, parser=lxml.etree.HTMLParser())
    if len(page.xpath('//*[@class="infobox-image"]/span/a')) < 1:
        if len(page.xpath('//*[@class="wikidata-claim"]/span/a')) < 1:
            if len(page.xpath('//*[@class="infobox"]/tr[3]/td/span/span/a')) < 1:
                # print(url + '\n', end='')
                return url

pool = ThreadPool(15)
results = list(pool.map(checkurl, df['WikiUrl']))
a = [i for i in results if i is not None]


In [17]:
b = [i + '_(город)' for i in a]
for i, v in enumerate(a):
    df.loc[df['WikiUrl'] == v, 'WikiUrl'] = b[i]

In [18]:
pool = ThreadPool(20)
results = list(pool.map(checkurl, b))

In [19]:
a = [i for i in results if i is not None]
for i, v in enumerate(a):
    df.loc[df['WikiUrl'] == v, 'WikiUrl'] = 'https://ru.wikipedia.org/w/index.php?search=' + \
                                            v.replace('https://ru.wikipedia.org/wiki/', '').replace('_(город)', '')

In [20]:
df

,Letter,City,Population,GoogleUrl,YandexUrl,WikiUrl
0,а,Астрахань,529793,https://www.google.com/search?q=Астрахань,https://yandex.ru/search/?text=Астрахань,https://ru.wikipedia.org/wiki/Астрахань
1,а,Архангельск,346979,https://www.google.com/search?q=Архангельск,https://yandex.ru/search/?text=Архангельск,https://ru.wikipedia.org/wiki/Архангельск
2,а,Ангарск,224630,https://www.google.com/search?q=Ангарск,https://yandex.ru/search/?text=Ангарск,https://ru.wikipedia.org/wiki/Ангарск
3,а,Армавир,188960,https://www.google.com/search?q=Армавир,https://yandex.ru/search/?text=Армавир,https://ru.wikipedia.org/wiki/Армавир_(город)
4,а,Абакан,186797,https://www.google.com/search?q=Абакан,https://yandex.ru/search/?text=Абакан,https://ru.wikipedia.org/wiki/Абакан
...,...,...,...,...,...,...
1099,я,Яранск,15475,https://www.google.com/search?q=Яранск,https://yandex.ru/search/?text=Яранск,https://ru.wikipedia.org/wiki/Яранск
1100,я,Ясногорск,15410,https://www.google.com/search?q=Ясногорск,https://yandex.ru/search/?text=Ясногорск,https://ru.wikipedia.org/w/index.php?search=Яс...
1101,я,Ясный,15249,https://www.google.com/search?q=Ясный,https://yandex.ru/search/?text=Ясный,https://ru.wikipedia.org/wiki/Ясный
1102,я,Яхрома,14056,https://www.google.com/search?q=Яхрома,https://yandex.ru/search/?text=Яхрома,https://ru.wikipedia.org/wiki/Яхрома


In [25]:
def checkurl(url):
    while True:
        try:
            s = requests.get(url).text
            page = lxml.etree.fromstring(s, parser=lxml.etree.HTMLParser())
            break
        except:
            pass
    if len(page.xpath('//*[@class="infobox-image"]/span/a')) < 1:
        if len(page.xpath('//*[@class="wikidata-claim"]/span/a')) < 1:
            if len(page.xpath('//*[@class="infobox"]/tr[3]/td/span/span/a')) < 1:
                try:
                    return page.xpath('//*[@class="image"]')[0].get('href')
                except:
                    print('4 ' + url + '\n', end='')
            else:
                try:
                    return page.xpath('//*[@class="infobox"]/tr[3]/td/span/span/a]')[0].get('href')
                except:
                    print('3 ' + url + '\n', end='')
                    try:
                        return page.xpath('//*[@class="image"]')[0].get('href')
                    except:
                        print('34 ' + url + '\n', end='')
        else:
            try:
                return page.xpath('//*[@class="wikidata-claim"]/span/a')[0].get('href')
            except:
                print('2 ' + url + '\n', end='')
                try:
                    return page.xpath('//*[@class="image"]')[0].get('href')
                except:
                    print('24 ' + url + '\n', end='')
    else:
        try:
            return page.xpath('//*[@class="infobox-image"]/span/a')[0].get('href')
        except:
            print('1 ' + url + '\n', end='')
            try:
                return page.xpath('//*[@class="image"]')[0].get('href')
            except:
                print('14 ' + url + '\n', end='')


pool = ThreadPool(10)
results = list(pool.map(checkurl, df['WikiUrl']))


4 https://ru.wikipedia.org/w/index.php?search=Дзержинск
4 https://ru.wikipedia.org/w/index.php?search=Димитровград
4 https://ru.wikipedia.org/w/index.php?search=Бор
4 https://ru.wikipedia.org/w/index.php?search=Волчанск
4 https://ru.wikipedia.org/w/index.php?search=Гурьевск
4 https://ru.wikipedia.org/w/index.php?search=Буинск
4 https://ru.wikipedia.org/w/index.php?search=Аксай
4 https://ru.wikipedia.org/w/index.php?search=Бабушкин
4 https://ru.wikipedia.org/w/index.php?search=Галич
4 https://ru.wikipedia.org/w/index.php?search=Белогорск
4 https://ru.wikipedia.org/w/index.php?search=Железногорск
4 https://ru.wikipedia.org/w/index.php?search=Березовский
4 https://ru.wikipedia.org/w/index.php?search=Куйбышев
4 https://ru.wikipedia.org/w/index.php?search=Кировск
4 https://ru.wikipedia.org/w/index.php?search=Краснознаменск
4 https://ru.wikipedia.org/w/index.php?search=Краснослободск
4 https://ru.wikipedia.org/w/index.php?search=Кременки
4 https://ru.wikipedia.org/w/index.php?search=Красноар

In [26]:
a = []
for i in results:
    if i is not None:
        if 'https' in i:
            print(i)
            a.append(None)
        else:
            a.append('https://ru.wikipedia.org' + i)
    else:
        a.append(None)
pd.DataFrame({
    'url': df['WikiUrl'],
    'pic': a
}).to_csv('123.csv')

https://classinform.ru/oktmo/search.php?str=92602101001
https://classinform.ru/oktmo/search.php?str=80703000001
https://classinform.ru/oktmo/search.php?str=28602101001
https://classinform.ru/oktmo/search.php?str=53708000001
https://classinform.ru/oktmo/search.php?str=65707000001
https://classinform.ru/oktmo/search.php?str=80606101001
https://classinform.ru/oktmo/search.php?str=73604101001
https://classinform.ru/oktmo/search.php?str=76606101001
https://classinform.ru/oktmo/search.php?str=53713000001
https://classinform.ru/oktmo/search.php?str=57615101001
https://classinform.ru/oktmo/search.php?str=65736000001
https://classinform.ru/oktmo/search.php?str=18608101001
https://classinform.ru/oktmo/search.php?str=38608101001
https://classinform.ru/oktmo/search.php?str=87608101001
https://classinform.ru/oktmo/search.php?str=15622101001
https://classinform.ru/oktmo/search.php?str=92627101001
https://classinform.ru/oktmo/search.php?str=10712000001
https://classinform.ru/oktmo/search.php?str=6071

In [27]:
# //*[@id="file"]/a
def download_pic(url):
    if url is not None:
        try:
            s = requests.get(url).text
        except:
            print('1 '+ url + '\n', end='')
            return None
        page = lxml.etree.fromstring(s, parser=lxml.etree.HTMLParser())
        try:
            return page.xpath('//*[@id="file"]/a')[0].get('href')
        except:
            print(url + '\n', end='')
            try:
                return page.xpath('//*[@class="image"]')[0].get('href')
            except:
                print('4 ' + url + '\n', end='')

pool = ThreadPool(50)
results = list(pool.map(download_pic, a))

https://ru.wikipedia.org/wiki/%D0%95%D0%BA%D0%B0%D1%82%D0%B5%D1%80%D0%B8%D0%BD%D0%B1%D1%83%D1%80%D0%B3-%D0%A1%D0%B8%D1%82%D0%B8
https://ru.wikipedia.org/wiki/%D0%92%D0%B5%D0%BB%D0%B8%D0%BA%D0%B8%D0%B9_%D0%9D%D0%BE%D0%B2%D0%B3%D0%BE%D1%80%D0%BE%D0%B4
https://ru.wikipedia.org/wiki/%D0%A0%D0%B8%D0%B3%D0%B0
https://ru.wikipedia.org/wiki/%D0%91%D0%B0%D1%88%D0%BD%D1%8F_%D0%A1%D1%8E%D1%8E%D0%BC%D0%B1%D0%B8%D0%BA%D0%B5
https://ru.wikipedia.org/wiki/%D0%94%D0%BC%D0%B8%D1%82%D1%80%D0%B8%D0%B5%D0%B2%D1%81%D0%BA%D0%B0%D1%8F_%D0%B1%D0%B0%D1%88%D0%BD%D1%8F
https://ru.wikipedia.org/w/index.php?title=%D0%92%D0%BE%D0%BA%D0%B7%D0%B0%D0%BB_%D0%9A%D1%80%D0%B0%D1%81%D0%BD%D0%BE%D0%B4%D0%B0%D1%80_I&action=edit&redlink=1
4 https://ru.wikipedia.org/w/index.php?title=%D0%92%D0%BE%D0%BA%D0%B7%D0%B0%D0%BB_%D0%9A%D1%80%D0%B0%D1%81%D0%BD%D0%BE%D0%B4%D0%B0%D1%80_I&action=edit&redlink=1
https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B0%D1%81%D1%81%D0%BA%D0%B0%D1%8F_%D0%B1%D0%B0%D1%88%D0%BD%D1%8F
https://ru.wikipedia.

In [28]:
a = []
for i in results:
    if i is not None:
        a.append('https:' + i)
    else:
        a.append(None)
c = pd.Series(a, name='PicUrl')
df = pd.concat([df, c], axis=1)
df

,Letter,City,Population,GoogleUrl,YandexUrl,WikiUrl,PicUrl
0,а,Астрахань,529793,https://www.google.com/search?q=Астрахань,https://yandex.ru/search/?text=Астрахань,https://ru.wikipedia.org/wiki/Астрахань,https://upload.wikimedia.org/wikipedia/commons...
1,а,Архангельск,346979,https://www.google.com/search?q=Архангельск,https://yandex.ru/search/?text=Архангельск,https://ru.wikipedia.org/wiki/Архангельск,https://upload.wikimedia.org/wikipedia/commons...
2,а,Ангарск,224630,https://www.google.com/search?q=Ангарск,https://yandex.ru/search/?text=Ангарск,https://ru.wikipedia.org/wiki/Ангарск,https://upload.wikimedia.org/wikipedia/commons...
3,а,Армавир,188960,https://www.google.com/search?q=Армавир,https://yandex.ru/search/?text=Армавир,https://ru.wikipedia.org/wiki/Армавир_(город),https://upload.wikimedia.org/wikipedia/commons...
4,а,Абакан,186797,https://www.google.com/search?q=Абакан,https://yandex.ru/search/?text=Абакан,https://ru.wikipedia.org/wiki/Абакан,https://upload.wikimedia.org/wikipedia/commons...
...,...,...,...,...,...,...,...
1099,я,Яранск,15475,https://www.google.com/search?q=Яранск,https://yandex.ru/search/?text=Яранск,https://ru.wikipedia.org/wiki/Яранск,https://upload.wikimedia.org/wikipedia/commons...
1100,я,Ясногорск,15410,https://www.google.com/search?q=Ясногорск,https://yandex.ru/search/?text=Ясногорск,https://ru.wikipedia.org/w/index.php?search=Яс...,None
1101,я,Ясный,15249,https://www.google.com/search?q=Ясный,https://yandex.ru/search/?text=Ясный,https://ru.wikipedia.org/wiki/Ясный,https://upload.wikimedia.org/wikipedia/commons...
1102,я,Яхрома,14056,https://www.google.com/search?q=Яхрома,https://yandex.ru/search/?text=Яхрома,https://ru.wikipedia.org/wiki/Яхрома,https://upload.wikimedia.org/wikipedia/commons...


In [29]:
# //*[@id="sidebar_content"]/div[2]/ul/li[1]/a
c = pd.Series(['https://www.openstreetmap.org/search?query=' + i for i in df['City']], name='MapUrl')
df = pd.concat([df, c], axis=1)
df

,Letter,City,Population,GoogleUrl,YandexUrl,WikiUrl,PicUrl,MapUrl
0,а,Астрахань,529793,https://www.google.com/search?q=Астрахань,https://yandex.ru/search/?text=Астрахань,https://ru.wikipedia.org/wiki/Астрахань,https://upload.wikimedia.org/wikipedia/commons...,https://www.openstreetmap.org/search?query=Аст...
1,а,Архангельск,346979,https://www.google.com/search?q=Архангельск,https://yandex.ru/search/?text=Архангельск,https://ru.wikipedia.org/wiki/Архангельск,https://upload.wikimedia.org/wikipedia/commons...,https://www.openstreetmap.org/search?query=Арх...
2,а,Ангарск,224630,https://www.google.com/search?q=Ангарск,https://yandex.ru/search/?text=Ангарск,https://ru.wikipedia.org/wiki/Ангарск,https://upload.wikimedia.org/wikipedia/commons...,https://www.openstreetmap.org/search?query=Анг...
3,а,Армавир,188960,https://www.google.com/search?q=Армавир,https://yandex.ru/search/?text=Армавир,https://ru.wikipedia.org/wiki/Армавир_(город),https://upload.wikimedia.org/wikipedia/commons...,https://www.openstreetmap.org/search?query=Арм...
4,а,Абакан,186797,https://www.google.com/search?q=Абакан,https://yandex.ru/search/?text=Абакан,https://ru.wikipedia.org/wiki/Абакан,https://upload.wikimedia.org/wikipedia/commons...,https://www.openstreetmap.org/search?query=Абакан
...,...,...,...,...,...,...,...,...
1099,я,Яранск,15475,https://www.google.com/search?q=Яранск,https://yandex.ru/search/?text=Яранск,https://ru.wikipedia.org/wiki/Яранск,https://upload.wikimedia.org/wikipedia/commons...,https://www.openstreetmap.org/search?query=Яранск
1100,я,Ясногорск,15410,https://www.google.com/search?q=Ясногорск,https://yandex.ru/search/?text=Ясногорск,https://ru.wikipedia.org/w/index.php?search=Яс...,None,https://www.openstreetmap.org/search?query=Ясн...
1101,я,Ясный,15249,https://www.google.com/search?q=Ясный,https://yandex.ru/search/?text=Ясный,https://ru.wikipedia.org/wiki/Ясный,https://upload.wikimedia.org/wikipedia/commons...,https://www.openstreetmap.org/search?query=Ясный
1102,я,Яхрома,14056,https://www.google.com/search?q=Яхрома,https://yandex.ru/search/?text=Яхрома,https://ru.wikipedia.org/wiki/Яхрома,https://upload.wikimedia.org/wikipedia/commons...,https://www.openstreetmap.org/search?query=Яхрома


In [30]:
# http://maps.google.com/maps/api/geocode/json?address=
# key = 'b8b4742f21550c630a8bd5d1f803a2b6'
# s = requests.get(f"http://api.openweathermap.org/geo/1.0/direct?q={'Perm'}&limit=1&appid={key}").text
# s

In [31]:
# //*[@id="search"]/table/tr[3]/td[2]/a[1]
# def coordinates(url):
#     if url is not None:
#         s = requests.get(url).text
#         page = lxml.etree.fromstring(s, parser=lxml.etree.HTMLParser())
#         try:
#             return page.xpath('//*[@id="search"]/table/tr[3]/td[2]/a')[0].get('href')
#         except:
#             return None
#
# a = list(df['MapUrl'])
#
# pool = ThreadPool(50)
# results = list(pool.map(coordinates, a))


In [32]:
# a = 0
# for i in results:
#     if i is None:
#         a += 1
# a, len(results)

In [33]:
# a = ['http://www.geonames.org' + i for i in results]


In [34]:
# def coordinates(url):
#     if url is not None:
#         s = requests.get(url).text
#         page = lxml.etree.fromstring(s, parser=lxml.etree.HTMLParser())
#         try:
#             return page.xpath('//*[@class="edit-mode move-mode"]')[0].Text, \
#                    page.xpath('//*[@class="edit-mode move-mode"]')[1].Text, s
#         except Exception as e:
#             return e, s
#
# b = [a[i] for i in range(100)]
# pool = ThreadPool(50)
# results = list(pool.map(coordinates, b))

In [35]:
# with open('123.html', 'w') as f:
#     f.write(results[0][1])
#
# results[0]

In [36]:
nomi = pgeocode.Nominatim('ru')
nomi.query_postal_code('Perm')

postal_code       PERM
country_code       NaN
place_name         NaN
state_name         NaN
state_code         NaN
county_name        NaN
county_code        NaN
community_name     NaN
community_code     NaN
latitude           NaN
longitude          NaN
accuracy           NaN
Name: 0, dtype: object

In [37]:
nom = nomi._data_frame
nom['accuracy'] = nom['accuracy'].astype('float')
nom = nom.sort_values('accuracy', ascending=False)
nom['place_name'] = nom['place_name'].str.lower()
nom.to_csv('123.csv')
nom

,country_code,postal_code,place_name,state_name,state_code,county_name,county_code,community_name,community_code,latitude,longitude,accuracy
0,RU,101000,москва,Москва,48.0,NaN,NaN,NaN,NaN,55.7522,37.6156,4.0
20097,RU,399583,гатище,Липецкая Область,43.0,ВОЛОВСКИЙ РАЙОН,NaN,NaN,NaN,52.2167,37.9442,4.0
20095,RU,399580,волово,Липецкая Область,43.0,ВОЛОВСКИЙ РАЙОН,NaN,NaN,NaN,52.0242,37.8769,4.0
20094,RU,399575,новопавловка,Липецкая Область,43.0,ВОЛОВСКИЙ РАЙОН,NaN,NaN,NaN,52.0264,38.7114,4.0
20093,RU,399574,васильевка,Липецкая Область,43.0,ВОЛОВСКИЙ РАЙОН,NaN,NaN,NaN,51.4853,40.2036,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
43533,RU,901985,а/т москва-владимир,Москва,48.0,NaN,NaN,NaN,NaN,55.7858,37.6256,NaN
43534,RU,901987,а/т москва-воронеж,Москва,48.0,NaN,NaN,NaN,NaN,55.7858,37.6256,NaN
43535,RU,901989,а/т москва-липецк,Москва,48.0,NaN,NaN,NaN,NaN,55.7858,37.6256,NaN
43536,RU,901991,а/т самара-ульяновск,Самарская Область,65.0,NaN,NaN,NaN,NaN,55.7858,37.6256,NaN


In [38]:
l = list(df['City'])
a = [nom.loc[nom['place_name'] == i.lower().replace('ё','е')] for i in l]

In [39]:
j = 0
for i in a:
    if len(i) == 0:
        j += 1
j

43

In [40]:
def f(i):
    try:
        return tuple(i.reset_index(drop=True).loc[0,['latitude', 'longitude']])
    except:
        return None

b = [f(i) for i in a]

In [41]:
nones = df.loc[[i for i, e in enumerate(b) if e is None], 'City']
nones

29               Алушта
35              Армянск
52               Алупка
96           Бахчисарай
217         Гаврилов-Ям
241             Джанкой
242         Дальнегорск
258             Долинск
265           Дмитровск
269           Евпатория
300                Зима
320              Зубцов
338            Инкерман
342           Иннополис
358               Керчь
377             Когалым
381        Красное Село
401             Кудрово
416          Кандалакша
433     Красноперекопск
459              Ковдор
475             Калязин
515           Ломоносов
518         Лесозаводск
563          Мончегорск
571          Муравленко
576          Медногорск
635               Надым
647            Невьянск
670           Нариманов
729            Петергоф
813         Севастополь
816         Симферополь
840         Североморск
844          Саяногорск
869                Саки
879             Сысерть
887               Судак
903         Старый Крым
1004           Феодосия
1024        Циолковский
1031         Чер

In [42]:
nom = Nominatim(user_agent="http")
loc = nom.geocode('пермь')
loc.raw

{'place_id': 258924502,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'relation',
 'osm_id': 3437213,
 'boundingbox': ['57.8678961', '58.1792804', '55.8042579', '56.6555926'],
 'lat': '58.0230679',
 'lon': '56.232803119037456',
 'display_name': 'Пермь, Пермский край, Приволжский федеральный округ, Россия',
 'class': 'place',
 'type': 'city',
 'importance': 0.35,
 'icon': 'https://nominatim.openstreetmap.org/ui/mapicons//poi_place_city.p.20.png'}

In [43]:
for i, v in nones.items():
    loc = nom.geocode('пермь')
    b[i] = (loc.raw['lat'], loc.raw['lon'])

In [44]:
nones = df.loc[[i for i, e in enumerate(b) if e is None], 'City']
nones

Series([], Name: City, dtype: object)

In [45]:
c = pd.Series([i[0] for i in b], name='Latitude')
df = pd.concat([df, c], axis=1)
df

,Letter,City,Population,GoogleUrl,YandexUrl,WikiUrl,PicUrl,MapUrl,Latitude
0,а,Астрахань,529793,https://www.google.com/search?q=Астрахань,https://yandex.ru/search/?text=Астрахань,https://ru.wikipedia.org/wiki/Астрахань,https://upload.wikimedia.org/wikipedia/commons...,https://www.openstreetmap.org/search?query=Аст...,46.34
1,а,Архангельск,346979,https://www.google.com/search?q=Архангельск,https://yandex.ru/search/?text=Архангельск,https://ru.wikipedia.org/wiki/Архангельск,https://upload.wikimedia.org/wikipedia/commons...,https://www.openstreetmap.org/search?query=Арх...,54.9
2,а,Ангарск,224630,https://www.google.com/search?q=Ангарск,https://yandex.ru/search/?text=Ангарск,https://ru.wikipedia.org/wiki/Ангарск,https://upload.wikimedia.org/wikipedia/commons...,https://www.openstreetmap.org/search?query=Анг...,52.53
3,а,Армавир,188960,https://www.google.com/search?q=Армавир,https://yandex.ru/search/?text=Армавир,https://ru.wikipedia.org/wiki/Армавир_(город),https://upload.wikimedia.org/wikipedia/commons...,https://www.openstreetmap.org/search?query=Арм...,44.98
4,а,Абакан,186797,https://www.google.com/search?q=Абакан,https://yandex.ru/search/?text=Абакан,https://ru.wikipedia.org/wiki/Абакан,https://upload.wikimedia.org/wikipedia/commons...,https://www.openstreetmap.org/search?query=Абакан,53.71
...,...,...,...,...,...,...,...,...,...
1099,я,Яранск,15475,https://www.google.com/search?q=Яранск,https://yandex.ru/search/?text=Яранск,https://ru.wikipedia.org/wiki/Яранск,https://upload.wikimedia.org/wikipedia/commons...,https://www.openstreetmap.org/search?query=Яранск,57.2994
1100,я,Ясногорск,15410,https://www.google.com/search?q=Ясногорск,https://yandex.ru/search/?text=Ясногорск,https://ru.wikipedia.org/w/index.php?search=Яс...,None,https://www.openstreetmap.org/search?query=Ясн...,54.4803
1101,я,Ясный,15249,https://www.google.com/search?q=Ясный,https://yandex.ru/search/?text=Ясный,https://ru.wikipedia.org/wiki/Ясный,https://upload.wikimedia.org/wikipedia/commons...,https://www.openstreetmap.org/search?query=Ясный,45.6139
1102,я,Яхрома,14056,https://www.google.com/search?q=Яхрома,https://yandex.ru/search/?text=Яхрома,https://ru.wikipedia.org/wiki/Яхрома,https://upload.wikimedia.org/wikipedia/commons...,https://www.openstreetmap.org/search?query=Яхрома,56.3


In [46]:
c = pd.Series([i[1] for i in b], name='Longitude')
df = pd.concat([df, c], axis=1)
df

,Letter,City,Population,GoogleUrl,YandexUrl,WikiUrl,PicUrl,MapUrl,Latitude,Longitude
0,а,Астрахань,529793,https://www.google.com/search?q=Астрахань,https://yandex.ru/search/?text=Астрахань,https://ru.wikipedia.org/wiki/Астрахань,https://upload.wikimedia.org/wikipedia/commons...,https://www.openstreetmap.org/search?query=Аст...,46.34,48.04
1,а,Архангельск,346979,https://www.google.com/search?q=Архангельск,https://yandex.ru/search/?text=Архангельск,https://ru.wikipedia.org/wiki/Архангельск,https://upload.wikimedia.org/wikipedia/commons...,https://www.openstreetmap.org/search?query=Арх...,54.9,59.74
2,а,Ангарск,224630,https://www.google.com/search?q=Ангарск,https://yandex.ru/search/?text=Ангарск,https://ru.wikipedia.org/wiki/Ангарск,https://upload.wikimedia.org/wikipedia/commons...,https://www.openstreetmap.org/search?query=Анг...,52.53,103.88
3,а,Армавир,188960,https://www.google.com/search?q=Армавир,https://yandex.ru/search/?text=Армавир,https://ru.wikipedia.org/wiki/Армавир_(город),https://upload.wikimedia.org/wikipedia/commons...,https://www.openstreetmap.org/search?query=Арм...,44.98,41.12
4,а,Абакан,186797,https://www.google.com/search?q=Абакан,https://yandex.ru/search/?text=Абакан,https://ru.wikipedia.org/wiki/Абакан,https://upload.wikimedia.org/wikipedia/commons...,https://www.openstreetmap.org/search?query=Абакан,53.71,91.42
...,...,...,...,...,...,...,...,...,...,...
1099,я,Яранск,15475,https://www.google.com/search?q=Яранск,https://yandex.ru/search/?text=Яранск,https://ru.wikipedia.org/wiki/Яранск,https://upload.wikimedia.org/wikipedia/commons...,https://www.openstreetmap.org/search?query=Яранск,57.2994,47.8822
1100,я,Ясногорск,15410,https://www.google.com/search?q=Ясногорск,https://yandex.ru/search/?text=Ясногорск,https://ru.wikipedia.org/w/index.php?search=Яс...,None,https://www.openstreetmap.org/search?query=Ясн...,54.4803,37.6975
1101,я,Ясный,15249,https://www.google.com/search?q=Ясный,https://yandex.ru/search/?text=Ясный,https://ru.wikipedia.org/wiki/Ясный,https://upload.wikimedia.org/wikipedia/commons...,https://www.openstreetmap.org/search?query=Ясный,45.6139,43.5381
1102,я,Яхрома,14056,https://www.google.com/search?q=Яхрома,https://yandex.ru/search/?text=Яхрома,https://ru.wikipedia.org/wiki/Яхрома,https://upload.wikimedia.org/wikipedia/commons...,https://www.openstreetmap.org/search?query=Яхрома,56.3,37.4833


In [47]:
wiki_wiki = wikipediaapi.Wikipedia('ru')
page_py = wiki_wiki.page('Астрахань')
page_py.summary

'А́страхань — город России, административный центр Астраханской области. Старейший экономический и культурный центр Нижнего Поволжья и Прикаспия. Входит в перечень исторических городов России. Образует городской округ город Астрахань. Центр Астраханской городской агломерации.\nРасполагается в верхней части дельты реки Волги, на 11 островах Прикаспийской низменности. Расстояние до Москвы по автодороге составляет 1550 км. Город разделён на 4 административных района: Кировский, Советский, Ленинский и Трусовский. Население: 524 371 чел. (2021). Площадь: 208,70 км². В Астрахани проживают представители более 200 национальностей, 19 конфессий, действуют 170 обществ национальных культур.'

In [48]:
def wiki(s):
    try:
        page_py = wiki_wiki.page(s).summary
        return page_py
    except:
        print(s + '\n', end="")
        return None

pool = ThreadPool(50)
results = list(pool.map(wiki,
                        [i.replace("https://ru.wikipedia.org/wiki/", "") for i in df['WikiUrl']]))

In [49]:
c = pd.Series(results, name='WikiSummary')
df = pd.concat([df, c], axis=1)
df

,Letter,City,Population,GoogleUrl,YandexUrl,WikiUrl,PicUrl,MapUrl,Latitude,Longitude,WikiSummary
0,а,Астрахань,529793,https://www.google.com/search?q=Астрахань,https://yandex.ru/search/?text=Астрахань,https://ru.wikipedia.org/wiki/Астрахань,https://upload.wikimedia.org/wikipedia/commons...,https://www.openstreetmap.org/search?query=Аст...,46.34,48.04,"А́страхань — город России, административный це..."
1,а,Архангельск,346979,https://www.google.com/search?q=Архангельск,https://yandex.ru/search/?text=Архангельск,https://ru.wikipedia.org/wiki/Архангельск,https://upload.wikimedia.org/wikipedia/commons...,https://www.openstreetmap.org/search?query=Арх...,54.9,59.74,Арха́нгельск — третий по численности город на ...
2,а,Ангарск,224630,https://www.google.com/search?q=Ангарск,https://yandex.ru/search/?text=Ангарск,https://ru.wikipedia.org/wiki/Ангарск,https://upload.wikimedia.org/wikipedia/commons...,https://www.openstreetmap.org/search?query=Анг...,52.53,103.88,"Анга́рск — город в Восточной Сибири, с прилега..."
3,а,Армавир,188960,https://www.google.com/search?q=Армавир,https://yandex.ru/search/?text=Армавир,https://ru.wikipedia.org/wiki/Армавир_(город),https://upload.wikimedia.org/wikipedia/commons...,https://www.openstreetmap.org/search?query=Арм...,44.98,41.12,"Армави́р — город на юге Российской Федерации, ..."
4,а,Абакан,186797,https://www.google.com/search?q=Абакан,https://yandex.ru/search/?text=Абакан,https://ru.wikipedia.org/wiki/Абакан,https://upload.wikimedia.org/wikipedia/commons...,https://www.openstreetmap.org/search?query=Абакан,53.71,91.42,Абака́н (хак. Ағбан) — город в России. Столица...
...,...,...,...,...,...,...,...,...,...,...,...
1099,я,Яранск,15475,https://www.google.com/search?q=Яранск,https://yandex.ru/search/?text=Яранск,https://ru.wikipedia.org/wiki/Яранск,https://upload.wikimedia.org/wikipedia/commons...,https://www.openstreetmap.org/search?query=Яранск,57.2994,47.8822,"Яра́нск — город в России, административный цен..."
1100,я,Ясногорск,15410,https://www.google.com/search?q=Ясногорск,https://yandex.ru/search/?text=Ясногорск,https://ru.wikipedia.org/w/index.php?search=Яс...,None,https://www.openstreetmap.org/search?query=Ясн...,54.4803,37.6975,
1101,я,Ясный,15249,https://www.google.com/search?q=Ясный,https://yandex.ru/search/?text=Ясный,https://ru.wikipedia.org/wiki/Ясный,https://upload.wikimedia.org/wikipedia/commons...,https://www.openstreetmap.org/search?query=Ясный,45.6139,43.5381,"Я́сный — город в Оренбургской области России, ..."
1102,я,Яхрома,14056,https://www.google.com/search?q=Яхрома,https://yandex.ru/search/?text=Яхрома,https://ru.wikipedia.org/wiki/Яхрома,https://upload.wikimedia.org/wikipedia/commons...,https://www.openstreetmap.org/search?query=Яхрома,56.3,37.4833,Я́хрома — город (с 1940) в Дмитровском городск...


In [50]:
df.loc[df['WikiSummary'] == ""]

,Letter,City,Population,GoogleUrl,YandexUrl,WikiUrl,PicUrl,MapUrl,Latitude,Longitude,WikiSummary
17,а,Аксай,46018,https://www.google.com/search?q=Аксай,https://yandex.ru/search/?text=Аксай,https://ru.wikipedia.org/w/index.php?search=Аксай,None,https://www.openstreetmap.org/search?query=Аксай,47.95,43.97,
68,б,Бор,76872,https://www.google.com/search?q=Бор,https://yandex.ru/search/?text=Бор,https://ru.wikipedia.org/w/index.php?search=Бор,None,https://www.openstreetmap.org/search?query=Бор,56.3567,44.0669,
73,б,Белогорск,65315,https://www.google.com/search?q=Белогорск,https://yandex.ru/search/?text=Белогорск,https://ru.wikipedia.org/w/index.php?search=Бе...,None,https://www.openstreetmap.org/search?query=Бел...,50.9158,128.4611,
78,б,Березовский,59030,https://www.google.com/search?q=Березовский,https://yandex.ru/search/?text=Березовский,https://ru.wikipedia.org/w/index.php?search=Бе...,None,https://www.openstreetmap.org/search?query=Бер...,45.47,43.33,
104,б,Буинск,20416,https://www.google.com/search?q=Буинск,https://yandex.ru/search/?text=Буинск,https://ru.wikipedia.org/w/index.php?search=Бу...,None,https://www.openstreetmap.org/search?query=Буинск,54.95,48.2833,
...,...,...,...,...,...,...,...,...,...,...,...
1001,у,Углегорск,8710,https://www.google.com/search?q=Углегорск,https://yandex.ru/search/?text=Углегорск,https://ru.wikipedia.org/w/index.php?search=Уг...,None,https://www.openstreetmap.org/search?query=Угл...,49.0833,142.0333,
1008,ф,Фокино,12759,https://www.google.com/search?q=Фокино,https://yandex.ru/search/?text=Фокино,https://ru.wikipedia.org/w/index.php?search=Фо...,None,https://www.openstreetmap.org/search?query=Фокино,57.4144,49.0486,
1045,ч,Чкаловск,11345,https://www.google.com/search?q=Чкаловск,https://yandex.ru/search/?text=Чкаловск,https://ru.wikipedia.org/w/index.php?search=Чк...,None,https://www.openstreetmap.org/search?query=Чка...,56.7647,43.2433,
1075,щ,Щучье,9476,https://www.google.com/search?q=Щучье,https://yandex.ru/search/?text=Щучье,https://ru.wikipedia.org/w/index.php?search=Щучье,None,https://www.openstreetmap.org/search?query=Щучье,51.7597,40.4875,


In [51]:
df.to_csv('Database/NewDatabase.csv')